In [1]:
%matplotlib inline

Uses Snow water equivalent from Margulis, S. A., G. Cortés, M. Girotto, and M. Durand (2016a), A Landsat-era Sierra Nevada (USA) snow reanalysis (1985–2015), J. Hydrometeorol., doi:10.1175/JHM-D-15-0177.1.

Aggregates each .h5 file (one for each year) according to a defined mask (included cells=1, otherwise no_data), matching the mask spatial resolution.

In [1]:
import os,sys
import numpy as np
import h5py
import netCDF4 as nc
import time
from datetime import datetime, timedelta, date
import calendar
from netCDF4 import num2date, date2num
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import iris
import cf_units

In [18]:
indir=os.path.normpath("X:/zraid_data5/ucla/snow/SWE")
maskdir=os.path.normpath("X:/projects/ucla/vic/gis")
outdir=os.path.normpath("X:/zraid_data5/ucla/snow")
mask="%s\%s" % (maskdir,"vicbasins_mask.nc")
start_year=2007
end_year=2015
V='SWE'

In [ ]:
#load mask
cube_mask = iris.load_cube(mask, 'mask')
cube_mask.coord('longitude').guess_bounds()
cube_mask.coord('latitude').guess_bounds()
print "Cube mask dims:"
print cube_mask.shape
cube_mask.data = np.ma.masked_less(cube_mask.data, 1)

# Open file. DATAFIELD_NAMEs found from gdalinfo <filename>
DATAFIELD_NAME = '//SWE'

for year in range(start_year, end_year+1):
    FILE_NAME="%s\SN_%s_WY%d.h5" % (indir,V,year)
    outf="%s\SN_%s_WY%d.nc" % (outdir,V,year)

    with h5py.File(FILE_NAME, mode='r') as f:
        # List available datasets.
        print f.keys()

        # Read dataset.
        dset = f[DATAFIELD_NAME]
        print dset
        print dset.shape #original coords are TXY
        ny=dset.shape[2]
        nx=dset.shape[1]
        nt=dset.shape[0]
        print nt,ny,nx

        #SET MASK DIMS TO NP ARRAY, THEN FILL WITH EACH DAY
        data_agg_3d = np.empty([dset.shape[0], cube_mask.shape[0], cube_mask.shape[1]], dtype='f')
        print "data_agg_shape:"
        print data_agg_3d.shape

        for j in range(nt):
            if (j+1)%30==0:
                print "day " + str(j+1) + " of " + str(nt)
            data = np.array(dset[j]) #extracting one day drops time dimension
            #original data has swapped x,y axes (SWE.axis('TXY')) - switch to typical CF TYX
            data = data.swapaxes(0, 1)
            data = np.where(data >= 0, data, np.nan)

            if j==0:
                #load lats and lons
                Latd = f['//lat']
                Lats = np.array(Latd[0,:])
                Lond = f['//lon']
                Lons = np.array(Lond[:,0])

                lat_coord = iris.coords.DimCoord(Lats,standard_name='latitude',units='degrees')
                lon_coord = iris.coords.DimCoord(Lons,standard_name='longitude',units='degrees')

            cube_data = iris.cube.Cube(data,dim_coords_and_dims=[(lat_coord, 0),(lon_coord, 1)])
            if not isinstance(cube_data.coord('longitude').bounds, np.ndarray):
                cube_data.coord('longitude').guess_bounds()
            if not isinstance(cube_data.coord('latitude').bounds, np.ndarray):
                cube_data.coord('latitude').guess_bounds()
            #couldn't get static regridder to work - this is not as efficient but works
            scheme = iris.analysis.AreaWeighted(mdtol=0.5)
            data_agg = cube_data.regrid(cube_mask, scheme)

            data_agg_3d[j,:,:]=data_agg.data

        try:
            dset.close()
        except:
            pass # Was already closed        

    #add time attribute and concatenate results into new cube, write
    dunits = 'days since %d-10-01 00:00:0.0' % (year-1)
    t_unit = cf_units.Unit(dunits,calendar='gregorian')
    time1 = iris.coords.DimCoord(np.arange(nt), standard_name='time', units=t_unit)
    lat_agg = cube_mask.coord('latitude')
    lon_agg = cube_mask.coord('longitude')
    agg_cube = iris.cube.Cube(data_agg_3d,dim_coords_and_dims=[(time1,0),(lat_agg, 1),(lon_agg, 2)], 
                              long_name='snow water equivalent', var_name='swe', units='mm')
    iris.save(agg_cube, outf,unlimited_dimensions='time')    

    print "Done writing %s" % (outf)
print "Done with all years"

Cube mask dims:
(128, 112)
[u'SWE', u'lat', u'lon']
<HDF5 dataset "SWE": shape (365, 5701, 6601), type "<i2">
(365, 5701, 6601)
365 6601 5701
data_agg_shape:
(365L, 128L, 112L)
day 30 of 365
day 60 of 365
day 90 of 365
day 120 of 365
day 150 of 365
day 180 of 365